<a href="https://colab.research.google.com/github/hoangnam-nguyen/sms_text_classification/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

### Read contents of files

In [ ]:
txt_train = open(train_file_path, 'rb').read().decode(encoding='utf-8')
txt_test = open(test_file_path, 'rb').read().decode(encoding='utf-8')

### Assess the datasets

Number of characters in each file

In [ ]:
print('Length of training dataset text: {} characters'.format(len(txt_train)))
print('Length of test dataset text: {} characters'.format(len(txt_test)))

How the txt_train looks like

In [ ]:
txt_train[:500]

Estimate the size of vocabulary in both training and test datasets

In [ ]:
text = (txt_train + '\n' + txt_test).replace('\n', ' ').replace('ham\t', '').replace('spam\t', '')
vocab = sorted(set(text))
vocab_size = len(vocab)

###Process the data

In [ ]:
txt_train = txt_train.split('\n')
txt_test = txt_test.split('\n')
print(len(txt_train))
print(len(txt_test))

Remove empty item in both lists

In [ ]:
txt_train = [x for x in txt_train if x != '']
txt_test = [x for x in txt_test if x != '']
print(len(txt_train))
print(len(txt_test))

Build training and test datasets (features and labels)

In [ ]:
from collections import Counter
from keras.preprocessing.text import one_hot
train_features = []
test_features = []
train_labels = []
test_labels = []

for line in txt_train:
  train_labels.append(line.split('\t')[0])
  train_features.append(one_hot(line.split('\t')[1], vocab_size))
for line in txt_test:
  test_labels.append(line.split('\t')[0])
  test_features.append(one_hot(line.split('\t')[1], vocab_size))

print(Counter(train_labels))
print(Counter(test_labels))

In [ ]:
train_labels = np.array([1 if label=='spam' else 0 for label in train_labels])
test_labels = np.array([1 if label=='spam' else 0 for label in test_labels])
print(train_features[0])
print(test_features[0])

Pad document to a max length of 150 (estimated from train_features)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_length = 150
train_padded = pad_sequences(train_features, maxlen=max_length)
test_padded = pad_sequences(test_features, maxlen=max_length)
print(train_padded[0])

###Build the model

Define model

In [ ]:
embedding_dim = 4
rnn_units = 8
epochs = 10

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim),
  tf.keras.layers.LSTM(rnn_units),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Compile model

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

Train model

In [ ]:
history = model.fit(train_padded, train_labels, epochs=epochs, validation_data=(test_padded, test_labels))

### Complete function to predict messages based on model

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  padded_text = pad_sequences([one_hot(pred_text, vocab_size)], maxlen=max_length)
  likeliness = model.predict(padded_text)[0][0]
  prediction = [likeliness, 'ham' if likeliness < 0.5 else 'spam']

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

### Pass freeCodeCamp's test

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
